<a href="https://colab.research.google.com/github/ryann-arruda/deep_learning_algorithms/blob/main/perceptron/pytorch_implementation/multiclass/perceptron_multiclass_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

## **Loading Dataset**

In [3]:
data = load_iris()

In [4]:
x = data.data

In [6]:
y = data.target

## **Data Normalization**

In [10]:
x = (x - x.min(axis=0))/(x.max(axis=0) - x.min(axis=0))

## **Separating the Dataset**

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=27)

## **Creating the Dataset Object**